<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/reprex/1-num_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

     |████████████████████████████████| 1.1MB 6.1MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 163kB 12.7MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 5.5MB 12.7MB/s 
     |████████████████████████████████| 645kB 46.6MB/s 
     |████████████████████████████████| 133kB 45.9MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 1.8MB 47.7MB/s 


In [4]:
import pyterrier as pt
pt.init()
# pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [5]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [6]:
N = 10_000

pt_index_path = './index_cord19'

indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True)

index_ref = indexer.index(
    (datum for i, datum in enumerate(dataset.get_corpus_iter()) if i < N),
    fields=('abstract',), # TODO should this be a list?
    # meta={'docno' : 26, 'text' : 2048, 'abstract' : 2048}
    meta=['docno','abstract'],
    )

index = pt.IndexFactory.of(index_ref)

[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
[INFO] [finished] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: [00:09] [269MB] [29.6MB/s]
[INFO] [starting] building docstore
docs_iter: 192509it [00:12, 15229.26it/s]
[INFO] [finished] docs_iter: [00:12] [192509it] [15227.61it/s]
[INFO] [finished] building docstore [12.65s]


07:20:42.146 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed

07:20:54.705 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 2143 empty documents


In [7]:
print(index.getCollectionStatistics().toString())

Number of documents: 10000
Number of terms: 36191
Number of postings: 673758
Number of fields: 1
Number of tokens: 1040094
Field names: [abstract]
Positions:   false



In [8]:
queries = dataset.get_topics(variant='title').head(3)
queries.head()

[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/0307a37b6b9f1a5f233340a769d538ea
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [6.71MB/s]


,qid,query
0,1,coronavirus origin
1,2,coronavirus response to weather changes
2,3,coronavirus immunity


In [13]:
M = 999

bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=M)

In [14]:
df_out = bm25.transform(queries)
df_out.head() # .loc[df_out["qid"] == "1", :]

,qid,docid,docno,rank,score,query
0,1,2257,8y24j34j,0,16.990326,coronavirus origin
1,1,4582,2jq626ye,1,12.327608,coronavirus origin
2,1,1394,vnafx1ng,2,8.248003,coronavirus origin
3,1,4980,cgby8bhz,3,7.913393,coronavirus origin
4,1,5341,qbt142yj,4,7.335607,coronavirus origin


In [15]:
df_out.loc[:, ["qid", "docid"]].groupby("qid").count()

,docid
qid,
1,353
2,999
3,999


In [16]:
M = 1002 # TODO is 1000 the max ever returned

bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=M)

bm25.transform(queries).loc[:, ["qid", "docid"]].groupby("qid").count()

,docid
qid,
1,353
2,1000
3,1000


In [21]:
# TODO why is this line commented out in source code?
# https://github.com/terrier-org/pyterrier/blob/a63a03c49b8b3d93e7b6a7a76f6f3c407264c918/pyterrier/batchretrieve.py#L114
bm25.appSetup.setProperty("matching.retrieved_set_size", str(M))

bm25.transform(queries).loc[:, ["qid", "docid"]].groupby("qid").count()

,docid
qid,
1,353
2,1002
3,1002


In [23]:
# TODO due to controls["end"], need to set both
# https://github.com/terrier-org/pyterrier/blob/a63a03c49b8b3d93e7b6a7a76f6f3c407264c918/pyterrier/batchretrieve.py#L113

# still gives max M
bm25.appSetup.setProperty("matching.retrieved_set_size", str(M + 100))

bm25.transform(queries).loc[:, ["qid", "docid"]].groupby("qid").count()

,docid
qid,
1,353
2,1002
3,1002


In [24]:
# creating a new instance correctly returns to the original, broken state
M = 988

bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=M)

bm25.transform(queries).loc[:, ["qid", "docid"]].groupby("qid").count()

,docid
qid,
1,353
2,988
3,988
